In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [3]:
#To import InstanceNormalization from keras-contrib
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-6doia83r
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-6doia83r
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=0feed905098a1f92f02d5b11c2257d93cc6874f1b70077507657299e5ec5241b
  Stored in directory: /tmp/pip-ephem-wheel-cache-k3ru5cj4/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [5]:
from keras.datasets import mnist
import numpy as np

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

condition_3 = np.where((Y_train == 3))
imgs_3= X_train[condition_3]

condition_6 = np.where((Y_train == 6))
imgs_6 = X_train[condition_6]


print('imgs-3 : {}'.format(imgs_3.shape))
print('imgs-6 : {}'.format(imgs_6.shape))


imgs-3 : (6131, 28, 28)
imgs-6 : (5918, 28, 28)


In [6]:
import cv2
import numpy as np
Train_3=[]
Train_6=[]

for img in imgs_3[:]:
  resize=cv2.resize(img, (32,32), fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
  normalize=resize/255.0
  Train_3.append(normalize)
  

  
for img in imgs_6[:]:
  resize=cv2.resize(img, (32,32), fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
  normalize=resize/255.0
  Train_6.append(normalize)
  


Train_3=np.array(Train_3).reshape(-1,32,32,1)  
Train_6=np.array(Train_6).reshape(-1,32,32,1)

print('Train-3 : {}'.format(Train_3.shape))
print('Train-6 : {}'.format(Train_6.shape))

Train-3 : (6131, 32, 32, 1)
Train-6 : (5918, 32, 32, 1)


In [28]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

import sys
import numpy as np
import os

# Input shape
img_rows = 32
img_cols = 32
channels = 1
img_shape = (img_rows, img_cols, channels)

# Calculate output shape of D (PatchGAN)
patch = int(img_rows/2**3)
disc_patch = (patch, patch, 1)

# Loss weights
lambda_cycle = 10.0               # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss

#Optimizer
optimizer = Adam(0.0002, 0.5)

def Generator():
  """Generator foloows U-Net Architecture"""

  # Image input
  inputs = Input(shape=img_shape)

  # Downsampling
  conv1 = Conv2D(filters=32, kernel_size=4, strides=2, padding='same')(inputs)
  conv1 = LeakyReLU(alpha=0.2)(conv1)
  conv1 = InstanceNormalization()(conv1)

  conv2 = Conv2D(filters=64, kernel_size=4, strides=2, padding='same')(conv1)
  conv2 = LeakyReLU(alpha=0.2)(conv2)
  conv2 = InstanceNormalization()(conv2)

  conv3 = Conv2D(filters=128, kernel_size=4, strides=2, padding='same')(conv2)
  conv3 = LeakyReLU(alpha=0.2)(conv3)
  conv3 = InstanceNormalization()(conv3)
  
  # Upsampling
  deconv1 = UpSampling2D(size=2)(conv3)
  deconv1 = Conv2D(filters=128, kernel_size=4, strides=1, padding='same', activation='relu')(deconv1)
  deconv1 = InstanceNormalization()(deconv1)
  deconv1 = Concatenate()([deconv1, conv2])

  deconv2 = UpSampling2D(size=2)(deconv1)
  deconv2 = Conv2D(filters=64, kernel_size=4, strides=1, padding='same', activation='relu')(deconv2)
  deconv2 = InstanceNormalization()(deconv2)
  deconv2 = Concatenate()([deconv2, conv1])

  deconv3 = UpSampling2D(size=2)(deconv2)
  outputs = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(deconv3)

  return Model(inputs = inputs, outputs = outputs)


def Discriminator():
  inputs = Input(shape=img_shape)
  
  outputs = Conv2D(filters=64, kernel_size=4, strides=2, padding='same')(inputs)
  outputs = LeakyReLU(alpha=0.2)(outputs)

  outputs = Conv2D(filters=128, kernel_size=4, strides=2, padding='same')(outputs)
  outputs = LeakyReLU(alpha=0.2)(outputs)
  outputs = InstanceNormalization()(outputs)

  outputs = Conv2D(filters=256, kernel_size=4, strides=2, padding='same')(outputs)
  outputs = LeakyReLU(alpha=0.2)(outputs)
  outputs = InstanceNormalization()(outputs)
  
  outputs = Conv2D(1, kernel_size=4, strides=1, padding='same')(outputs)

  return Model(inputs = inputs, outputs = outputs)

# Build and compile the discriminators
D_A = Discriminator()
D_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
print(D_A.summary())

D_B = Discriminator()
D_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

# Build the generators
G_AB = Generator()
print(G_AB.summary())
G_BA = Generator()

# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

# Translate images to the other domain
fake_B = G_AB(img_A)
fake_A = G_BA(img_B)
# Translate images back to original domain
reconstr_A = G_BA(fake_B)
reconstr_B = G_AB(fake_A)
# Identity mapping of images
img_A_id = G_BA(img_A)
img_B_id = G_AB(img_B)

# For the combined model we will only train the generators
D_A.trainable = False
D_B.trainable = False

# Discriminators determines validity of translated images
valid_A = D_A(fake_A)
valid_B = D_B(fake_B)

# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B], outputs=[valid_A, valid_B, reconstr_A, reconstr_B, img_A_id, img_B_id ])
combined.compile(loss=['mse', 'mse', 'mae', 'mae', 'mae', 'mae'], loss_weights=[1, 1, lambda_cycle, lambda_cycle, lambda_id, lambda_id], optimizer=optimizer)


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 16, 16, 64)        1088      
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 8, 8, 128)         131200    
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
instance_normalization_31 (I (None, 8, 8, 128)         2         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 4, 4, 256)         524

In [30]:
import datetime
import sys
import matplotlib.pyplot as plt
import random

os.chdir('/content/drive/My Drive/GitHub Repositories/Generative Models Papers with Implementation in Keras')
baseDir = './CycleGAN-Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks'
outputDir = os.path.join(baseDir, 'MNIST outputs')
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
    print('Output Directory Created to save Results')

start_time = datetime.datetime.now()
epochs = 5
batch_size = 1
batches=int(5500/batch_size)
save_interval = 200
#limitize the training data to first 5500 images and use rest for test data
limit = 5500
# Adversarial loss ground truths
valid = np.ones((batch_size,) + disc_patch)
fake = np.zeros((batch_size,) + disc_patch)




def save_imgs(epoch, batch):
  # pick random image from test data based on random indexes and reshape the image to tensor
  index_A = random.randint(limit, Train_3.shape[0])
  imgs_A = Train_3[index_A]
  imgs_A = imgs_A.reshape(-1, img_rows, img_cols, channels)

  index_B = random.randint(limit, Train_6.shape[0])
  imgs_B = Train_6[index_B]
  imgs_B = imgs_B.reshape(-1, img_rows, img_cols, channels)

  # Translate images to opposite domain
  fake_B = G_AB.predict(imgs_A)
  fake_A = G_BA.predict(imgs_B)  
  
  # Translate back to original domain
  reconstr_A = G_BA.predict(fake_B)
  reconstr_B = G_AB.predict(fake_A)

  gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  titles = ['Original-3', 'Translated-6', 'Reconstructed-3', 'Original-6', 'Translated-3', 'Reconstructed-6']
  fig=plt.figure(1)
  for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(gen_imgs[i,..., 0], cmap='gray')
    plt.gca().set_title(titles[i])
    plt.axis('off')
  plt.show()
  fig.savefig(outputDir+'/image_at_epoch_{:04d}_batch_{:04d}.png'.format(epoch, batch))
  plt.close()

# Training 
for epoch in range(epochs):
  epoch+=1
  for batch in range(batches):
    batch+=1
    # ----------------------
    #  Train Discriminators
    # ----------------------

    # pick random image from training data based on random indexes and reshape the image to tensor
    index_A = random.randint(0, limit)
    imgs_A = Train_3[index_A]
    imgs_A = imgs_A.reshape(-1, img_rows, img_cols, channels)

    index_B = random.randint(0, limit)
    imgs_B = Train_6[index_B]
    imgs_B = imgs_B.reshape(-1, img_rows, img_cols, channels)


    # Translate images to opposite domain
    fake_B = G_AB.predict(imgs_A)
    fake_A = G_BA.predict(imgs_B)
    
    # Train the discriminators (original images = real / translated = Fake)
    dA_loss_real = D_A.train_on_batch(imgs_A, valid)
    dA_loss_fake = D_A.train_on_batch(fake_A, fake)
    dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

    dB_loss_real = D_B.train_on_batch(imgs_B, valid)
    dB_loss_fake = D_B.train_on_batch(fake_B, fake)
    dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

    # Total disciminator loss
    d_loss = 0.5 * np.add(dA_loss, dB_loss)
    
    # ------------------
    #  Train Generators
    # ------------------

    # Train the generators
    g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, valid, imgs_A, imgs_B, imgs_A, imgs_B])

    elapsed_time = datetime.datetime.now() - start_time
    sys.stdout.write('\r Epoch: {0}/{1} | Batch: {2}/{3} | D-loss: {4:.3f}, D-acc: {5:.2f}% | G-loss: {6:.3f} | adv: {7:.3f} | recon: {8:.3f} | id: {9:.3f} | time: {10}'.
                     format(epoch, epochs, batch, batches, d_loss[0], 100*d_loss[1], g_loss[0], np.mean(g_loss[1:3]), np.mean(g_loss[3:5]),
                            np.mean(g_loss[5:6]), elapsed_time))
    # After specific number of iterations say 200 in our case save results
    # If at save interval => save generated image samples
    if batch % save_interval == 0:
      save_imgs(epoch, batch)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'CycleGAN-MNIST-training-outputs.gif'), images, fps = 3)